# Superstores Analyses

## 1. Le dataset

In [1]:
#import chardet
#with open('superstore.csv', 'rb') as rawdata:
#    result = chardet.detect(rawdata.read(100000))
#result


# On importe les modules nécessaires
import numpy as np
import pandas as pd
import datatable as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_dark"

# lecture du fichier csv et on l'applique à df
df = pd.read_csv('superstore.csv', sep=',', encoding="Windows-1252", index_col=0)

# Transformation des variables temporelles en type datetime
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df['Order Date'] = pd.to_datetime(df['Order Date'])
df = df.drop(['Postal Code', 'Country'], axis=1) # On supprime ces variables car non pertinentes

# On affiche les informations sur les colonnes ainsi qu'une partie du dataframe
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9994 entries, 1 to 9994
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       9994 non-null   object        
 1   Order Date     9994 non-null   datetime64[ns]
 2   Ship Date      9994 non-null   datetime64[ns]
 3   Ship Mode      9994 non-null   object        
 4   Customer ID    9994 non-null   object        
 5   Customer Name  9994 non-null   object        
 6   Segment        9994 non-null   object        
 7   City           9994 non-null   object        
 8   State          9994 non-null   object        
 9   Region         9994 non-null   object        
 10  Product ID     9994 non-null   object        
 11  Category       9994 non-null   object        
 12  Sub-Category   9994 non-null   object        
 13  Product Name   9994 non-null   object        
 14  Sales          9994 non-null   float64       
 15  Quantity       9994 n

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
# infos statistiques sur les variables quantitatives
df.describe()

In [ ]:
desc = []
for col in df.columns.to_list():
    if df[col].nunique() < 18:
        desc.append(''.join(str(df[col].unique())))
    elif df[col].nunique() > 18:
        desc.append("Too many")
        
  
#df = df.drop(['u', 'Orders'], axis=1)


fig = go.Figure(data=[go.Table(
    columnwidth=[150, 100, 600], 
    header=dict(values=['Column', 
                        'Modes', 
                        'Description']),
                 cells=dict(values=[df.columns.to_list(), 
                                   [df[col].nunique() for col in df.columns.to_list()], 
                                   desc], 
                           align=('center', 'center', 'left'), 
                           fill=dict(color=['lightsteelblue', 'lightslategrey', 'lightsteelblue']), 
                           height = 30, 
                           font=dict(color='black', size=12)))
                     ])
fig.update_layout(width=1000, height=900, template='plotly_dark')
fig.show()

## Variables numériques
> Ci-dessous on trace la distribution des variables quantitatives

In [ ]:
# Affichage de la distribution des variables numériques
fig = make_subplots(
    rows=3, cols=1,
    row_heights=[0.3, 0.3, 0.4],
    specs=[[{"type" : "box"}],
           [{"type" : "box"}],
           [{"type" : "box"}]],
    subplot_titles=("Distribution de la variable Sales", 
                    "Distribution de la variable Profit", 
                    "Distribution de la variable Discount"))
fig.add_trace(go.Box(x=df['Sales'], marker_color = 'royalblue', name='Sales'), 1, 1)
fig.add_trace(go.Box(x=df['Profit'], marker_color = 'lightseagreen', name='Profit'), 2, 1)
fig.add_trace(go.Box(x=df['Discount'], marker_color = 'lightcoral', name='Discount'), 3, 1)
fig.update_layout(height=800, template='plotly_dark')
fig.show()

> Sales : cette variable a quelques valeurs extrêmes lorsqu'on dépasse 5k d'achats  
> Profit : le profit varie de -6k à 8k environ.  
> Discount : cette variable détient des valeurs comprises entre 0 et 80% de réduction. La médiane se situant à 20%.  

In [ ]:
df = df.sort_values('Order Date')

# On scinde le dataset par année
df_2014 = df[df['Order Date'] <= '2014-12-31']
df_2014 = df_2014.groupby('Order Date').mean()
df_2015 = df[(df['Order Date'] >= '2015-01-01') & (df['Order Date'] <= '2015-12-31')]
df_2015 = df_2015.groupby('Order Date').mean()
df_2016 = df[(df['Order Date'] >= '2016-01-01') & (df['Order Date'] <= '2016-12-31')]
df_2016 = df_2016.groupby('Order Date').mean()
df_2017 = df[(df['Order Date'] >= '2017-01-01') & (df['Order Date'] <= '2017-12-31')]
df_2017 = df_2017.groupby('Order Date').mean()
df_2018 = df[(df['Order Date'] >= '2018-01-01') & (df['Order Date'] <= '2018-12-31')]
df_2018 = df_2018.groupby('Order Date').mean()

# Affichage de la plus grosse perte d'argent
#df[df['Order Date'] == '2015-01-28']

Après avoir fait la moyenne des ventes par jour, on affiche la variation de Sales et Profit sur le même graphique :

In [ ]:
# On regroupe les ventes par jour, puis on trace sales et profit dans le temps
# mean
df_tot = df.groupby('Order Date').agg({'Sales':'mean', 'Profit':'mean'})
df_tot = df_tot.sort_values('Order Date', ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_tot.index, y = df_tot['Sales'], line = dict(color='red', width=2), name = 'Sales'))
fig.add_trace(go.Scatter(x = df_tot.index, y = df_tot['Profit'], line = dict(color='green', width=2), name = 'Profit'))
fig.update_traces(overwrite=True)
fig.update_layout(title = 'Variation de Sales & Profit au cours du temps : moyenne', template='plotly_dark')
fig.show()

> On observe certaines valeurs négatives en termes de profit. Il serait intéressant de découvrir pourquoi.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df['Discount'][df.Profit < -1000], y=df['Profit'][df.Profit < -1000]))
fig.update_layout(barmode='stack', title_text='Profit vs Discount', xaxis_title='Discount', yaxis_title='Profit', template='plotly_dark')
fig.show()

En effet, les plus grosses promotions induisent les plus grosses pertes de profit.

In [ ]:
# profit vs discount across category
fig = px.bar(x=df['Discount'][df.Profit < -1000], y=df['Profit'][df.Profit < -1000], color=df['Category'][df.Profit < -1000])
fig.update_layout(barmode='stack', 
                  title_text='Profit vs Discount', 
                  xaxis_title='Discount', 
                  yaxis_title='Profit', 
                  legend_title='Category', 
                  template='plotly_dark')
fig.show()

En affichant les pertes par catégorie, on remarque que les plus grosses pertes sont issues des ventes de fournitures de bureau.

In [ ]:
# profit vs discount across region
fig = px.bar(x=df['Discount'][df.Profit < -1000], y=df['Profit'][df.Profit < -1000], color=df['Region'][df.Profit < -1000])
fig.update_layout(barmode='stack', title_text='Profit vs Discount', xaxis_title='Discount', yaxis_title='Profit', legend_title='Region')
fig.show()

# profit vs category across sub-category
fig = px.bar(x=df['Category'][df.Profit < -1000], y=df['Profit'][df.Profit < -1000], color=df['Sub-Category'][df.Profit < -1000])
fig.update_layout(barmode='stack', title_text='Profit vs Category', xaxis_title='Category', yaxis_title='Profit', legend_title='Sub-Category')
fig.show()

Profit vs Discount :
> On voit que les pertes se concentrent sur la région centrale des Etats-Unis, ainsi que la région Est.  

Profit vs Category
> Ce graphique nous informe que ce sont les classeurs et machines qui constituent les plus grandes pertes.

## Variables catégorielles
On décide maintenant d'afficher la distribution des variables catégorielles :

In [ ]:
# subplots de la distribution de variables qualitatives
fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Histogram(x=df['Ship Mode'], name='Ship Mode', marker_color='darkblue', histnorm='percent'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Segment'], name='Segment', histnorm='percent'), row=1, col=2)
fig.add_trace(go.Histogram(x=df['Category'], name='Category', histnorm='percent'), row=2, col=1)
fig.add_trace(go.Histogram(x=df['Region'], name='Region', histnorm='percent'), row=2, col=2)

fig.update_layout(title = 'Répartition des variables catégorielles',
                 yaxis_title = 'Density',
                 width = 1000, 
                 height = 600)
                 #template='simple_white')
fig.show()

# distribution de la variable sub-category
fig = go.Figure()
fig.add_trace(go.Histogram(x=df['Sub-Category'], name = 'Sub-Category', histnorm='percent', nbinsx=len(df['Sub-Category'].unique())))
fig.update_layout(title = 'Sub-Category',
                 yaxis_title = 'Density',
                 width = 1000, 
                 height = 400,
                 #template='simple_white',
                 xaxis={'categoryorder':'total descending'})
fig.show()

> __Ship Mode__ : sans surprise, la classe standard est la préférée des clients.  
> __Segment__ : Le plus gros segment est représenté par les particuliers, vient ensuite les sociétés puis le home office.  
> __Catgory__ : La catégorie d'objet qui se vend le plus est les fournitures de bureau, puis le mobilier et les objets technologiques en dernier.  
> __Region__ : La région Ouest enregistre le plus de ventes.

Traçons maintenant la relation entre les ventes et les profits selon les catégories :

In [ ]:
# scatter plot de sales et profit
fig = go.Figure()
fig = px.scatter(x=df.Sales, y=df.Profit, trendline='ols', color=df.Category)
fig.update_layout(title='Sales vs Profit', yaxis_title='Profit', xaxis_title='Sales', legend_title='Category')
fig.show()

La catégorie de produits technologiques offre les profits les plus importants.

## Scatter Geo

Après avoir récolter les données géographiques de chaque ville, on les intègre au dataframe afin d'obtenir les latitudes et longitudes pour chaque ville

In [ ]:
gf = pd.read_csv('uscities.csv', index_col='id')
gf = gf.drop(gf.loc[gf['state_name'] == 'Hawaii'].index.to_list())
gf = gf.drop(gf.loc[gf['state_name'] == 'Alaska'].index.to_list())
gf = gf.drop(gf.loc[gf['state_name'] == 'Puerto Rico'].index.to_list())
gf = gf.drop(gf.loc[gf['city'] == 'New Hempstead'].index.to_list())
gf = gf.drop(gf.loc[gf['city'] == 'West Hempstead'].index.to_list())
gf = gf[['city', 'state_name', 'lat', 'lng']]

gf['city'].loc[gf['city'] == 'New York'] = 'New York City'
gf['city'].loc[gf['city'] == 'Fairfield University'] = 'Fairfield'
gf['city'].loc[gf['city'] == 'East Orange'] = 'Orange'
gf['city'].loc[gf['city'] == 'South Hempstead'] = 'Hempstead'
gf['city'].loc[gf['city'] == 'McAllen'] = 'Mcallen'
gf['city'].loc[gf['city'] == 'Milford city'] = 'Milford'
gf.loc[1840002792, 'city'] = 'East Orange'

city_list = []
for city in gf['city']:
    if 'St.' in city:
        city_list.append(city.replace('St.', 'Saint'))
    else:
        city_list.append(city)  
gf['city'] = city_list

gf.info()
gf.head()

On les intègre maintenant à notre dataframe global, en faisant attention aux doublons ou différences orthographiques.

In [ ]:
# Création du dictionnaire avec pour chaque ville la latitude et la longitude
import time
start = time.time()
d = dict()
for i in range(gf.shape[0]):
    if gf['state_name'].iloc[i] not in d:
        d[gf['state_name'].iloc[i]] = dict()
    d[gf['state_name'].iloc[i]].update({gf['city'].iloc[i] : (gf['lat'].iloc[i], gf['lng'].iloc[i])})

end = time.time()

# On modifie les entrées dans le dataset pour matcher avec celles du dictionnaire
d['New Jersey'].update({'Belleville' : (43.7893, -76.1153)})
d['New Jersey'].update({'Lakewood' : (42.0992, -79.3201)})
d['Connecticut'].update({'Manchester' : (41.780434, -72.541466)})
d['Massachusetts'].update({'Franklin' : (42.0834, -71.3967)})
d['Michigan'].update({'Canton' : (42.308644, -83.482116)})
d['New Jersey'].update({'Bridgeton' : (39.429564, -75.230461)})
d['Connecticut'].update({'Milford' : (41.230698, -73.064034)})
d['Massachusetts'].update({'Andover' : (42.658336, -71.136795)}) 
d['New Jersey'].update({'Orange' : (40.7651, -74.2117)}) 

lat = []
lng = []
for n in range(df.shape[0]):
    lat.append(d[df['State'].iloc[n]][df['City'].iloc[n]][0])
    lng.append(d[df['State'].iloc[n]][df['City'].iloc[n]][1])

df['lat'] = lat
df['lng'] = lng
print(f'performed in : {end-start} seconds')

On effectue au préalable un groupby qui permet d'afficher les moyennes des ventes par ville

In [ ]:
lt = []
ln = []
ml = pd.DataFrame(df.groupby(['State', 'City']).mean())

for n in range(ml.shape[0]):
    lt.append(d[ml.index[n][0]][ml.index[n][1]][0])
    ln.append(d[ml.index[n][0]][ml.index[n][1]][1])

ml['lat'] = lt
ml['lng'] = ln
ml.head()

In [ ]:
cities = [ml.index[i][1] for i in range(ml.shape[0])] # permet d'avoir une liste contenant le nom des villes

**Scatter Geo** nous permet d'afficher pour chaque ville, les variables qu'on souhaite, sous forme de bulles dont la taille est définie par la variable choisie.

In [ ]:
fig = px.scatter_geo(
    ml,
    lat=ml['lat'],
    lon=ml['lng'],
    size = ml['Sales'],
    hover_name = cities,
    color = ml['Profit'],
    width=1000,
    height=800
)

fig.update_layout(
        title = 'Average sales by city across profit',
        geo_scope ='usa',
    )
fig.show()

On distingue bien les villes qui obtiennent les meilleurs profits, telles que : Jamestown, Lafayette, Independance...

# KPI

Attelons-nous à afficher certains KPIs tel que le panier moyen par exemple :

In [ ]:
# panier moyen = CA / nombre de commandes 
#df['Orders'] = np.ones(df.shape[0], int)
hf = df.groupby('Customer ID').agg({'Sales':'mean', 'Profit':'mean', 'Orders':'sum'}).reset_index()
hf = hf.sort_values('Customer ID', ascending=True)
bf = df.groupby('City').agg({'Sales':'mean', 'Profit':'mean', 'Orders':'sum'}).reset_index()
bf = bf.sort_values('City', ascending=True)

hf['avg basket'] = hf['Sales'] / hf['Orders']
bf['avg basket'] = bf['Sales'] / bf['Orders']

fig = go.Figure()
fig.add_trace(go.Bar(x=hf['Customer ID'], y=hf['avg basket']))
fig.update_layout(title='Average basket per customer', xaxis_title='Customer ID', yaxis_title='Average basket')
fig.update_xaxes(tickangle=-60)
fig.show()

fig = go.Figure()
fig.add_trace(go.Bar(x=bf['City'], y=bf['avg basket']))
fig.update_layout(title='Average basket per city', xaxis_title='City', yaxis_title='Average basket')
fig.update_xaxes(tickangle=-60)
fig.show()

# Analyse statistique

On peut en premier lieu, utiliser le test de _Pearson_, qui nous informe sur la corrélation entre variables quantitatives :

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True,cmap='viridis');
plt.title('Coefficient de Pearson');

> Il existe aussi le test _ANOVA_, qui permet de déterminer la corrélation entre variables numériques et catégorielles.

In [ ]:
import statsmodels.api
result = statsmodels.formula.api.ols('Profit ~ Category', data=df).fit()
table = statsmodels.api.stats.anova_lm(result)
print('ANOVA TEST : Profit vs Category')
table

Le test ANOVA calcule 2 valeurs importantes : La statistique F et la p-value. Plus F est élevée moins les variables sont corrélées. Inversement proportionnelle avec la p-value.

Enfin, le test du Chi 2 permet d'analyser les variables catégorielles et mettre en évidence leur corrélation.

In [ ]:
from scipy.stats import chi2_contingency
df_cat = df.select_dtypes('object')
def V_Cramer(table, N):
    stat_chi2 = chi2_contingency(table)[0]
    k = table.shape[0]
    r = table.shape[1]
    phi_2 = max(0,(stat_chi2)/N - ((k - 1)*(r - 1)/(N - 1)))
    k_b = k - (np.square(k - 1) / (N - 1))
    r_b = r - (np.square(r - 1) / (N - 1))   
    return np.sqrt(phi_2 / min(k_b - 1, r_b - 1))


dico = {}
for col in df_cat.columns[df_cat.columns != 'Sales']:
    table = pd.crosstab(df_cat[col], df['Sales'])
    res = chi2_contingency(table)
    dico[col] = [res[0], res[1], res[2], V_Cramer(table, df.shape[0])]
    
    
stats = pd.DataFrame.from_dict(dico).transpose()
stats = stats.rename(columns={0:'$\chi 2$', 1:'p-value', 2:'DoF', 3:'V de Cramer'})
stats